In [ ]:
import pandas as pd # pandas for data manipulation
import numpy as np # numpy for numerical calculation
import panel as pn # panel  library  is used  for making   dashboard
pn.extension('tabulator') 
import hvplot.pandas # hvplot for pandas

In [ ]:
df = pd.read_csv("/content/GCB2022v27_percapita_flat.csv")  # read_csv funtion is used for loading data
df

In [ ]:
df.isnull().sum() #  checking null value

In [ ]:
df.dropna(inplace=True)# dropna function is used for drop  null values

In [ ]:
df.columns # checking all columns present in our data set

In [ ]:
df.corr() # corr is used for checking corelation

In [ ]:
df.isnull().sum()

In [ ]:
df['Country'].unique() #unique function is used for finding unique values

In [ ]:
df[df['Country']=='Netherlands'] # only those value print which is erqual to Netherlands 

In [ ]:
df[df['Country']=='USA']

In [ ]:
idf = df.interactive() # initates  the slider

In [ ]:
df['Year'].unique().min() # find the min year 

In [ ]:
df['Year'].unique().max() # find the max year  which is present 

In [ ]:
slider_year = pn.widgets.IntSlider(name='Year slider', start=1994, end=2021, step=2, value=1950) # making slider usinf Intslider function which is present  in panel libery 
slider_year

In [ ]:
#  making Radio button
radio_button = pn.widgets.RadioButtonGroup(
    name='Radio Button Group', options=['Oil', 'Gas', ], button_type='success'
    )


radio_button

In [ ]:
continents = ['USA', 'Netherlands','Austria','Turkey','Italy']

In [ ]:
co2_pipeline = (
    idf[
        (idf.Year <= slider_year) &(idf.Country.isin(continents)) # creating pipline 
    ].groupby(['Country', 'Year'])[radio_button].mean().to_frame().reset_index().sort_values(by='Year') .reset_index(drop=True)
)

In [ ]:
co2_pipeline

In [ ]:
co2_plot = co2_pipeline.hvplot(x = 'Year', by='Country', y=radio_button,line_width=2, title="CO2 emission  Year vs Country")
co2_plot

##  Making table  

In [ ]:
table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width')  # making table and connect to slide bar and radio button  
table

##  Making  scatterplot

In [ ]:
scatterplot_pipeline = (
    idf[
        (idf.Year == slider_year) &
        (~ (idf.Country.isin(continents)))
    ]
    .groupby(['Country', 'Year', 'Oil'])['Gas'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')  
    .reset_index(drop=True)
)

In [ ]:
scatterplot_pipeline

In [ ]:
scatterplot =scatterplot_pipeline.hvplot(x='Oil', 
                                                                y='Gas', 
                                                                by='Country', 
                                                                size=80, kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                height=500, 
                                                                width=500)
scatterplot

In [ ]:
second_radio_button = pn.widgets.RadioButtonGroup( # making radiobutton 
    name='Y axis', 
    options=['Gas', 'Oil', 'Coal'], 
    button_type='success'
)

continents = ['USA', 'Netherlands','Austria','Turkey','Italy']

co2_source_bar_pipeline = ( # making pipline 
    idf[
        (idf.Year == slider_year) &
        (idf.Country.isin(continents))
    ]
    .groupby(['Year', 'Country'])[second_radio_button].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')  
    .reset_index(drop=True)
)

In [ ]:
bar_plot = co2_source_bar_pipeline.hvplot(kind='bar', 
                                                     x='Country', # making bar chart 
                                                     y=second_radio_button, 
                                                     title='CO2 source by continent')
bar_plot

In [ ]:
hist_plot = co2_source_bar_pipeline.hvplot(kind='hist',  # making histogram 
                                                     x='Country', 
                                                     y=second_radio_button, 
                                                     title='CO2 source by continent')
hist_plot

In [ ]:
box_plot = co2_source_bar_pipeline.hvplot(kind='box',  # Making Boxplot 
                                                     x='Country', 
                                                     y=second_radio_button, 
                                                     title='CO2 source by continent')
box_plot

## Creating Dashboard

In [ ]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='World CO2 emission dashboard', 
    sidebar=[pn.pane.Markdown("# CO2 Emissions and Climate Change"), 
             pn.pane.Markdown("#### Carbon dioxide (CO2) emissions are a major contributor to climate change. Climate change refers to long-term changes in the Earth's climate, including global warming, changes in precipitation patterns, sea level rise, and more frequent and intense extreme weather events such as heat waves, floods, and droughts."), 
            pn.pane.Markdown("#### CO2 emissions are mainly caused by the burning of fossil fuels such as coal, oil, and natural gas, which release carbon dioxide into the atmosphere. This increase in atmospheric CO2 levels leads to a greenhouse effect, where heat is trapped in the Earth's atmosphere, causing the Earth's temperature to rise."), 
            pn.pane.Markdown("#### Climate change has far-reaching impacts on the natural environment, as well as on human societies and economies. It can affect agriculture, water resources, human health, and biodiversity. In addition, climate change can exacerbate poverty, inequality, and social instability, particularly in vulnerable communities and developing countries."), 
            pn.pane.Markdown("#### To mitigate the impacts of climate change, efforts are being made to reduce CO2 emissions and transition to cleaner energy sources such as renewable energy. This includes policies and regulations to limit emissions, investment in low-carbon technologies, and behavior changes at the individual level."), 
            pn.pane.Markdown("#### Overall, reducing CO2 emissions is essential to mitigating the impacts of climate change and ensuring a sustainable future for the planet and its inhabitants."), 

             pn.pane.Markdown("## Settings"),
             pn.pane.Markdown("## about"),

             slider_year],
    main=[pn.Row(pn.Column(radio_button, 
                           co2_plot.panel(width=700), margin=(0,25)), 
                 table.panel(width=500)), 
          pn.Row(pn.Column(scatterplot.panel(width=600), margin=(0,25)),
                  pn.pane.Markdown("### scatter plots to visually examine the relationship between two variables, identify patterns and trends"),
                  
                 pn.Column(second_radio_button, bar_plot.panel(width=600)),
                  pn.pane.Markdown("## We use bar plots to compare and display the frequency, count, or percentage of categorical data in different categories, making it easy to identify differences and similarities between groups or variables."),),
          pn.Row(pn.Column(second_radio_button, hist_plot.panel(width=600)),
           pn.pane.Markdown("## A histogram is a graphical representation of the distribution of a continuous variable, showing the frequency or count of data points falling within each interval, and providing insight into the shape, center, and spread of a datase"),
                 pn.Column(second_radio_button, box_plot.panel(width=600)),
                  pn.pane.Markdown("## A box plot is a graphical representation of the distribution of a dataset, showing the median, quartiles, range, and outliers, making it easy to summarize and compare different datasets."),)],
     accent_base_color="#00FFFF",
    header_background="#800080",
)
# template.show()
template.servable();

In [ ]:
template.show()